#### Python Version

In [1]:
from platform import python_version
print(python_version())

#ToDO - Need VM with python 3.10

3.10.12


#### Install Libraries

In [2]:
# pip install ydata_profiling dtale
from IPython.display import clear_output

! pip install openai
! pip install chromadb

clear_output()

#### Formatting

In [4]:
bold_s = "\033[1m"
bold_e = "\033[0m"

#### Imports and set Path

In [6]:
import chromadb
from chromadb.config import Settings

from vertexai.language_models import TextEmbeddingModel

#### Global params

In [14]:
CHROMA_DB_PATH = 'gs://roughdraft/villages/emopti/workshops/llm_patient_search/workers/chroma_vector_db/chroma_openai'
EMBEDDINGS_MODEL_NAME = "text-embedding-ada-002"
CHROMA_DB_LOCAL_FOLDER_NAME = "chroma_openai"
COLLECTION_NAME = "emopti_search"

print(f"EMBEDDINGS_PATH: {CHROMA_DB_PATH}")
print(f"EMBEDDINGS_MODEL_NAME: {EMBEDDINGS_MODEL_NAME}")
print(f"CHROMA_DB_LOCAL_FOLDER_NAME: {CHROMA_DB_LOCAL_FOLDER_NAME}")
print(f"COLLECTION_NAME: {COLLECTION_NAME}")
print()

#### Copy DB to local folder
! gsutil -m cp -r {CHROMA_DB_PATH} CHROMA_DB_LOCAL_FOLDER_NAME


EMBEDDINGS_PATH: gs://roughdraft/villages/emopti/workshops/llm_patient_search/workers/chroma_vector_db/chroma_openai
EMBEDDINGS_MODEL_NAME: text-embedding-ada-002
CHROMA_DB_LOCAL_FOLDER_NAME: chroma_openai
COLLECTION_NAME: emopti_search



CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: 6 files/objects could not be transferred.


#### Chroma DB

In [15]:
# PersistentClient
print(f"\nCreate Chroma Client and Collection")
client = chromadb.PersistentClient(path= CHROMA_DB_LOCAL_FOLDER_NAME, settings=Settings(allow_reset=True))
collection = client.get_or_create_collection(name=COLLECTION_NAME, metadata={"hnsw:space": "cosine"})

# print(f"\nlist of the first 10 items in the collection: {collection.peek()}")
print(f"\nnumber of items in the collection: {collection.count()}")



Create Chroma Client and Collection

number of items in the collection: 10000


#### Query DB

In [18]:
# Test query
# Test query
patient_info = """Complaint: 'cramping pain;"""

embedding_model = TextEmbeddingModel.from_pretrained(EMBEDDINGS_MODEL_NAME)
test_embeddings = embedding_model.get_embeddings([patient_info])[0].values

search_results = collection.query(
    query_embeddings=[test_embeddings],
    n_results=5
)

print(f"\n{bold_s}Search Patient{bold_e}: {patient_info}")

print(f"\nTop Matched Patients: \n")

for id, distance in zip(search_results['ids'][0], search_results['distances'][0]):
        
    print(f"{bold_s}ID{bold_e}: {id}")
    print(f"{bold_s}Similariry{bold_e}: {(1-distance):.4f}")
    print()



Search Patient: Complaint: 'cramping pain;

Top Matched Patients: 

ID: 6c0a3480a8f7c8a1c5c552cfe2adddfad6a3b7c15f6fa839cf2df9370badd364
Similariry: 0.8340

ID: b2b0e44c052944ac55b4136de913a75bedc5a8aa31dbd4c36fc66b08a1e2f656
Similariry: 0.8314

ID: 9de4bab565738986855321f3fed7b7e4d91daff43024924f0820862a4cde9866
Similariry: 0.8314

ID: 0899222f45307e430cbd2d2f886e698aba2af86a880665e526bdf16bcb5231e2
Similariry: 0.8301

ID: 71fd3e9f934d4179371425994bdbc46a15a520eb5e29186e856453fa8fd45b82
Similariry: 0.8288

